In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Настройка для отображения
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Библиотеки успешно импортированы")


Библиотеки успешно импортированы


## 1. Анализ структуры файлов


In [2]:
# Определение путей к данным
data_path = Path("extracted_data")
csv_path = data_path / "csv" / "syn_csv"
json_path = data_path / "json" / "syn_json"
xml_path = data_path / "xml" / "syn_xml"

# Подсчет файлов в каждой папке
csv_files = list(csv_path.glob("*.csv"))
json_files = list(json_path.glob("*.json"))
xml_files = list(xml_path.glob("*.xml"))

print(f"CSV файлов: {len(csv_files)}")
print(f"JSON файлов: {len(json_files)}")
print(f"XML файлов: {len(xml_files)}")

# Анализ размеров файлов
def get_file_sizes(file_list):
    sizes = []
    for file in file_list:
        size_mb = file.stat().st_size / (1024 * 1024)
        sizes.append(size_mb)
    return sizes

csv_sizes = get_file_sizes(csv_files)
json_sizes = get_file_sizes(json_files)
xml_sizes = get_file_sizes(xml_files)

print(f"\nРазмеры CSV файлов (MB):")
print(f"  Минимум: {min(csv_sizes):.2f}")
print(f"  Максимум: {max(csv_sizes):.2f}")
print(f"  Средний: {np.mean(csv_sizes):.2f}")
print(f"  Общий размер: {sum(csv_sizes):.2f}")

print(f"\nРазмеры JSON файлов (MB):")
print(f"  Минимум: {min(json_sizes):.2f}")
print(f"  Максимум: {max(json_sizes):.2f}")
print(f"  Средний: {np.mean(json_sizes):.2f}")
print(f"  Общий размер: {sum(json_sizes):.2f}")

print(f"\nРазмеры XML файлов (MB):")
print(f"  Минимум: {min(xml_sizes):.2f}")
print(f"  Максимум: {max(xml_sizes):.2f}")
print(f"  Средний: {np.mean(xml_sizes):.2f}")
print(f"  Общий размер: {sum(xml_sizes):.2f}")


CSV файлов: 12
JSON файлов: 31
XML файлов: 15

Размеры CSV файлов (MB):
  Минимум: 332.02
  Максимум: 332.34
  Средний: 332.18
  Общий размер: 3986.16

Размеры JSON файлов (MB):
  Минимум: 239.46
  Максимум: 414.72
  Средний: 267.59
  Общий размер: 8295.36

Размеры XML файлов (MB):
  Минимум: 20.28
  Максимум: 958.83
  Средний: 786.73
  Общий размер: 11800.91


## 2. Анализ CSV данных


In [3]:
# Загрузка первого CSV файла для анализа структуры
sample_csv = csv_files[0]
print(f"Анализируем файл: {sample_csv.name}")

# Читаем только первые несколько строк для анализа структуры, устойчиво к проблемным строкам и разделителям
try:
    df_sample = pd.read_csv(
        sample_csv,
        nrows=1000,
        sep=None,              # авто-определение разделителя
        engine='python',       # более терпимый парсер
        on_bad_lines='skip',   # пропускать битые строки
        quoting=3              # QUOTE_NONE
    )
except Exception as e:
    print(f"Не удалось прочитать CSV стандартным способом: {e}\nПробую с запятой как разделителем...")
    df_sample = pd.read_csv(
        sample_csv,
        nrows=1000,
        sep=',',
        engine='python',
        on_bad_lines='skip'
    )

print(f"\nРазмер выборки: {df_sample.shape}")
print(f"\nКолонки:")
for i, col in enumerate(df_sample.columns):
    print(f"  {i+1}. {col}")

print(f"\nТипы данных:")
print(df_sample.dtypes)

print(f"\nПервые 5 строк:")
display(df_sample.head())

print(f"\nБазовая статистика:")
display(df_sample.describe(include='all'))


Анализируем файл: part-00000-37dced01-2ad2-48c8-a56d-54b4d8760599-c000.csv

Размер выборки: (950, 27)

Колонки:
  1. created
  2. order_status
  3. ticket_status
  4. ticket_price
  5. visitor_category
  6. event_id
  7. is_active
  8. valid_to
  9. count_visitor
  10. is_entrance
  11. is_entrance_mdate
  12. event_name
  13. event_kind_name
  14. spot_id
  15. spot_name
  16. museum_name
  17. start_datetime
  18. ticket_id
  19. update_timestamp
  20. client_name
  21. name
  22. surname
  23. client_phone
  24. museum_inn
  25. birthday_date
  26. order_number
  27. ticket_number

Типы данных:
created               object
order_status          object
ticket_status         object
ticket_price         float64
visitor_category      object
event_id               int64
is_active               bool
valid_to              object
count_visitor          int64
is_entrance             bool
is_entrance_mdate     object
event_name            object
event_kind_name       object
spot_id           

,created,order_status,ticket_status,ticket_price,visitor_category,event_id,is_active,valid_to,count_visitor,is_entrance,is_entrance_mdate,event_name,event_kind_name,spot_id,spot_name,museum_name,start_datetime,ticket_id,update_timestamp,client_name,name,surname,client_phone,museum_inn,birthday_date,order_number,ticket_number
0,2021-08-18T16:01:14.583+03:00,PAID,PAID,0.0,Обучающиеся по очной форме обучения в государс...,7561,True,2021-08-18,1,True,2021-08-18T19:14:45.427+03:00,Бальный танец,"консультация, методическое занятие, семинар, т...",274010,"Шверника ул. 13, корпус 2",Государственное бюджетное учреждение культуры ...,2021-08-18 17:00:00,1778482,2021-08-18T16:01:15.682+03:00,ШУКУРОВ РУСЛАН,КИРИЛЛ,ШУКУРОВ,7.985948e+10,3832203597,NaN,75343-483088,07a16922-969c-1033-a23f-f20b57dcf045
1,2021-09-16T18:45:54.401+03:00,PAID,PAID,250.0,Взрослые граждане РФ и СНГ,1765,True,2021-09-18,1,False,2021-09-16T18:45:54.977+03:00,Посещение Дворца (Большого дома),выставка,69,Усадьба Кусково,Государственное бюджетное учреждение культуры ...,2021-09-18 10:00:00,2100707,2021-09-16T18:46:19.713+03:00,КОБЕЛЬКОВА АЛЁНА,СЕРГЕЙ,КОБЕЛЬКОВ,7.926477e+10,415706141,NaN,04091-683930,2d6fcdd1-c16d-9501-bda8-ae329e98a5ca
2,2021-05-06T12:02:04.808+03:00,PAID,PAID,800.0,Взрослые граждане РФ и СНГ,1473,True,2021-05-06,1,True,2021-05-06T12:03:08.989+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-06 07:30:00,881269,2021-05-06T12:02:05.566+03:00,БОБРОВ ЮРИЙ,NaN,NaN,NaN,8719976264,NaN,87693-505050,06c34a5c-b6b7-1b92-b245-460c1a7ce3e9
3,2021-05-25T16:05:06.883+03:00,PAID,PAID,0.0,"Студенты, обучающиеся по очной форме обучения ...",1473,True,2021-05-25,1,True,2021-05-25T17:15:54.616+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-25 07:30:00,1147456,2021-05-25T16:05:07.733+03:00,КРЕБСА СОФИЯ,ИЛЬЯ,КРЕБС,7.911649e+10,1678121766,NaN,36421-684592,22f9764a-a036-2fb8-ac3a-2c98a58784b9
4,2021-05-07T14:49:50.473+03:00,PAID,PAID,0.0,Дети до 7 лет (не достигшие семилетнего возраста),1473,True,2021-05-07,1,False,2021-05-07T14:49:50.822+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-07 07:30:00,907812,2021-05-07T14:50:59.624+03:00,ПЕТРОСЯНА ГАЛИНА,ДМИТРИЙ,ПЕТРОСЯН,7.977650e+10,2599957179,NaN,72297-435006,441c1a65-1d3a-3e8a-85a4-be47e023f563



Базовая статистика:


,created,order_status,ticket_status,ticket_price,visitor_category,event_id,is_active,valid_to,count_visitor,is_entrance,is_entrance_mdate,event_name,event_kind_name,spot_id,spot_name,museum_name,start_datetime,ticket_id,update_timestamp,client_name,name,surname,client_phone,museum_inn,birthday_date,order_number,ticket_number
count,950,950,950,950.00000,950,950.000000,950,950,950.0,950,950,950,950,950.000000,950,950,923,9.500000e+02,950,950,684,684,7.270000e+02,9.500000e+02,165,950,950
unique,949,2,2,NaN,28,NaN,1,274,NaN,2,950,124,17,NaN,70,47,473,NaN,949,949,245,634,NaN,NaN,165,950,950
top,2021-09-30T16:15:53.622+03:00,PAID,PAID,NaN,Взрослые граждане РФ и СНГ,NaN,True,2021-08-22,NaN,True,2021-08-18T19:14:45.427+03:00,Посещение Московского зоопарка,выставка,NaN,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-07 07:30:00,NaN,2021-09-30T16:16:43.409+03:00,ЕФИМОВА МАРИЯ,АЛЕКСАНДР,ИВАНОВА,NaN,NaN,1989-10-28,75343-483088,07a16922-969c-1033-a23f-f20b57dcf045
freq,2,944,944,NaN,540,NaN,950,18,NaN,737,1,598,859,NaN,598,598,12,NaN,2,2,29,7,NaN,NaN,1,1,1
mean,NaN,NaN,NaN,321.80000,NaN,2004.008421,NaN,NaN,1.0,NaN,NaN,NaN,NaN,207230.994737,NaN,NaN,NaN,1.241817e+06,NaN,NaN,NaN,NaN,7.923116e+10,4.930066e+09,NaN,NaN,NaN
std,NaN,NaN,NaN,367.72708,NaN,1492.883859,NaN,NaN,0.0,NaN,NaN,NaN,NaN,119484.123682,NaN,NaN,NaN,6.601487e+05,NaN,NaN,NaN,NaN,7.041278e+08,2.864902e+09,NaN,NaN,NaN
min,NaN,NaN,NaN,0.00000,NaN,587.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,3.910200e+04,NaN,NaN,NaN,NaN,7.165523e+10,2.546349e+06,NaN,NaN,NaN
25%,NaN,NaN,NaN,0.00000,NaN,1473.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,271246.000000,NaN,NaN,NaN,6.561912e+05,NaN,NaN,NaN,NaN,7.910514e+10,2.491375e+09,NaN,NaN,NaN
50%,NaN,NaN,NaN,50.00000,NaN,1473.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,276345.000000,NaN,NaN,NaN,1.274764e+06,NaN,NaN,NaN,NaN,7.919925e+10,4.847933e+09,NaN,NaN,NaN
75%,NaN,NaN,NaN,800.00000,NaN,1963.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,276345.000000,NaN,NaN,NaN,1.816621e+06,NaN,NaN,NaN,NaN,7.958409e+10,7.314322e+09,NaN,NaN,NaN


## 3. Анализ JSON данных


In [4]:
# Анализ структуры JSON файла
sample_json = json_files[0]
print(f"Анализируем файл: {sample_json.name}")

# Читаем JSON файл
with open(sample_json, 'r', encoding='utf-8') as f:
    # Читаем только первые несколько строк для анализа структуры
    lines = []
    for i, line in enumerate(f):
        if i >= 10:  # Берем только первые 10 строк
            break
        lines.append(line.strip())

print(f"\nПервые 10 строк JSON файла:")
for i, line in enumerate(lines):
    print(f"{i+1}: {line[:200]}..." if len(line) > 200 else f"{i+1}: {line}")

# Пытаемся загрузить один JSON объект для анализа структуры
try:
    with open(sample_json, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        json_obj = json.loads(first_line)
        
    print(f"\nСтруктура JSON объекта:")
    if isinstance(json_obj, dict):
        print(f"Ключи: {list(json_obj.keys())}")
        for key, value in json_obj.items():
            print(f"  {key}: {type(value).__name__} - {str(value)[:100]}...")
    else:
        print(f"Тип данных: {type(json_obj).__name__}")
        print(f"Содержимое: {str(json_obj)[:200]}...")
        
except Exception as e:
    print(f"Ошибка при анализе JSON: {e}")


Анализируем файл: part1.json

Первые 10 строк JSON файла:
1: [
2: {
3: "date_exec": "2024-10-09",
4: "code_form_ind_entrep": "1",
5: "name_form_ind_entrep": "Индивидуальный предприниматель",
6: "inf_surname_ind_entrep_sex": "1",
7: "citizenship_kind": "1",
8: "inf_authority_reg_ind_entrep_name": "Управление Федеральной налоговой службы по Пензенской области",
9: "inf_authority_reg_ind_entrep_code": "5800",
10: "inf_reg_tax_ind_entrep": "Управление Федеральной налоговой службы по Пензенской области",
Ошибка при анализе JSON: Expecting value: line 1 column 2 (char 1)


## 4. Анализ XML данных


In [5]:
# Анализ структуры XML файла
sample_xml = xml_files[0]
print(f"Анализируем файл: {sample_xml.name}")

try:
    # Читаем XML файл
    tree = ET.parse(sample_xml)
    root = tree.getroot()
    
    print(f"\nКорневой элемент: {root.tag}")
    print(f"Атрибуты корневого элемента: {root.attrib}")
    
    # Анализируем структуру
    def analyze_xml_structure(element, level=0, max_level=3):
        if level > max_level:
            return
        
        indent = "  " * level
        print(f"{indent}{element.tag}")
        
        if element.attrib:
            print(f"{indent}  Атрибуты: {element.attrib}")
        
        if element.text and element.text.strip():
            text = element.text.strip()[:100]
            print(f"{indent}  Текст: {text}...")
        
        # Анализируем дочерние элементы (только первые 5)
        children = list(element)[:5]
        for child in children:
            analyze_xml_structure(child, level + 1, max_level)
        
        if len(list(element)) > 5:
            print(f"{indent}  ... и еще {len(list(element)) - 5} элементов")
    
    print(f"\nСтруктура XML (первые 3 уровня):")
    analyze_xml_structure(root)
    
except Exception as e:
    print(f"Ошибка при анализе XML: {e}")
    
    # Если не удается распарсить как XML, читаем как текст
    try:
        with open(sample_xml, 'r', encoding='utf-8') as f:
            content = f.read(1000)  # Читаем первые 1000 символов
            print(f"\nПервые 1000 символов файла:")
            print(content)
    except Exception as e2:
        print(f"Не удалось прочитать файл: {e2}")


Анализируем файл: part1.xml

Корневой элемент: root
Атрибуты корневого элемента: {}

Структура XML (первые 3 уровня):
root
  item
    actual_unverified_data
      purpose
    dated_info
      oti
    metadata
      last_change_record_number
        Текст: 50:21:0140218:1614-11...
      last_container_fixed_at
        Текст: 2019-01-09T00:00:00+03:00...
      status
        Текст: actual...
    object_common_data
      cad_number
        Текст: 50:21:0140218:1614...
      previously_posted
        Текст: true...
      quarter_cad_number
        Текст: 77:17:0140116...
      type
    object_formation
      method
    ... и еще 2 элементов
  item
    actual_unverified_data
      purpose
    dated_info
      oti
    metadata
      last_change_record_number
        Текст: 50:21:0140207:1336-24...
      last_container_fixed_at
        Текст: 2021-12-26T12:46:59+00:00...
      status
        Текст: actual...
    object_common_data
      cad_number
        Текст: 50:21:0140207:1336...
      pr